# Installations

Installation Function to Page wise Image Conversion

In [ ]:
# %pip install pymupdf

Installation Function to Cropping Tables

In [ ]:
# %%capture
# %pip install "unstructured[all-docs]"

installations for Function to Recognise and format tables as csv

In [ ]:
# %pip install paddlepaddle paddleocr tensorflow pandas

# Imports

In [ ]:
import os
import pandas as pd
import fitz
import cv2
from unstructured.partition.image import partition_image
import numpy as np
import tensorflow as tf
from paddleocr import PaddleOCR


# Define Directories

In [ ]:
input_dir = "PDF STORE"
output_dir = "TEST RESULT"
deb_output_dir = "DEBUG_IMGs"

# Image Extraction

In [ ]:
def extract_images_from_pdf(input_dir, output_base_dir="RESULTS/IMAGES"):
    # ... existing code ...
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(".pdf"):
            # Build the full path to the PDF file
            pdf_path = os.path.join(input_dir, filename)
            pdf_name = os.path.splitext(filename)[0]

            # Create the corresponding output directory
            output_folder = os.path.join(output_base_dir, f"{pdf_name}-images")
            os.makedirs(output_folder, exist_ok=True)

            # Open the PDF file
            pdf_document = fitz.open(pdf_path)
            print(f"Processing PDF: {pdf_path}")

            # Iterate over each page in the PDF
            for page_number in range(pdf_document.page_count):
                page = pdf_document[page_number]
                images = page.get_images(full=True)
                print(f"  Page {page_number + 1} has {len(images)} image(s).")

                # Extract and save each image
                for img_index, img in enumerate(images):
                    xref = img[0]
                    base_image = pdf_document.extract_image(xref)
                    image_bytes = base_image["image"]

                    # Construct a filename for each image
                    image_path = os.path.join(
                        output_folder,
                        f"Page_{page_number + 1}_Image_{img_index + 1}.png"
                    )

                    # Save the image to a file
                    with open(image_path, "wb") as img_file:
                        img_file.write(image_bytes)
                    print(f"    Saved: {image_path}")

            # Close the PDF document
            pdf_document.close()
    
    print(f"\nAll PDFs processed successfully! Images are stored in: {output_base_dir}")

input_directory = "TEST INPUT PDF"
extract_images_from_pdf(input_directory)

# Unstructured Tables Extraction (OCR Based)

Extracting Text from PDF - Acepted -- Done

In [ ]:
def extract_text_from_pdf(input_dir, text_output_folder="Extracted Text", dpi=300):
    """
    Extract text directly from PDF files without storing intermediate page images.
    """
    os.makedirs(text_output_folder, exist_ok=True)
    
    # Get all PDF files from input directory
    pdf_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.pdf')]

    # Process each PDF file
    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_dir, pdf_file)
        pdf_name = os.path.splitext(pdf_file)[0]
        
        # Create output folder for this PDF's text
        pdf_text_output_folder = os.path.join(text_output_folder, f"{pdf_name}-Texts")
        os.makedirs(pdf_text_output_folder, exist_ok=True)

        print(f"\nProcessing PDF: {pdf_file}")
        
        # Open PDF
        pdf_document = fitz.open(pdf_path)

        # Process each page
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            
            # Convert PDF page to image
            pix = page.get_pixmap(dpi=dpi, colorspace=fitz.csRGB)
            
            # Convert to numpy array for OpenCV
            img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, 3)
            
            # Convert RGB to BGR for OpenCV
            image_cv = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
            
            page_name = f'page_{page_number + 1}'
            
            # Create a temporary file for the image
            temp_image_path = os.path.join(text_output_folder, "temp_page.png")
            cv2.imwrite(temp_image_path, image_cv)
            
            try:
                # Detect text using Unstructured's partition_image with filename
                elements = partition_image(filename=temp_image_path, infer_table_structure=True, strategy='hi_res')
                
                # Convert elements to JSON structure
                element_dict = [el.to_dict() for el in elements]
                
                # Extract text if type is not "Table"
                for item in element_dict:
                    if isinstance(item, dict) and item.get("type") != "Table":
                        text_content = item.get("text", "")
                        if text_content:
                            text_filename = f"{page_name}_text.txt"
                            text_output_path = os.path.join(pdf_text_output_folder, text_filename)
                            with open(text_output_path, "a", encoding="utf-8") as text_file:
                                text_file.write(text_content + "\n")
                            print(f"Text extracted and saved to: {text_output_path}")
            
            finally:
                # Clean up temporary file
                if os.path.exists(temp_image_path):
                    os.remove(temp_image_path)
            
            print(f"Processed page {page_number + 1}/{pdf_document.page_count}")
        
        pdf_document.close()
        print(f"Finished processing PDF: {pdf_file}! Text saved in: {pdf_text_output_folder}")

    print(f"\nAll PDFs processed successfully! Extracted text is stored in: {text_output_folder}")

# Example usage:
input_directory = "TEST INPUT PDF"  # Directory containing PDF files
text_output_directory = "TEST RESULT3/Extracted Text"  # Directory for output text

extract_text_from_pdf(input_directory, text_output_directory)

Cropping Tables - Accepted -- Done

In [ ]:
def crop_tables_from_pdf(input_dir, output_base_folder="Extracted Tables", dpi=300, top_left_padding=5, bottom_right_padding=7):
    """
    Extract tables directly from PDF files without storing intermediate page images.
    """
    os.makedirs(output_base_folder, exist_ok=True)
    
    # Get all PDF files from input directory
    pdf_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.pdf')]

    # Function to crop an image with padding and save it
    def crop_with_padding(image, coordinates, output_folder, filename):
        """Crop the specified coordinates from the image, apply padding, and save it."""
        x_min = int(min(pt[0] for pt in coordinates))
        y_min = int(min(pt[1] for pt in coordinates))
        x_max = int(max(pt[0] for pt in coordinates))
        y_max = int(max(pt[1] for pt in coordinates)) 

        # Apply top-left and bottom-right padding
        x_min_padded = max(0, x_min - top_left_padding)
        y_min_padded = max(0, y_min - top_left_padding)
        x_max_padded = min(image.shape[1], x_max + bottom_right_padding * 2)
        y_max_padded = min(image.shape[0], y_max + bottom_right_padding)

        # Crop the image
        cropped_image = image[y_min_padded:y_max_padded, x_min_padded:x_max_padded]

        # Save cropped image
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, cropped_image)
        print(f"Cropped table saved to: {output_path}")

    # Process each PDF file
    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_dir, pdf_file)
        pdf_name = os.path.splitext(pdf_file)[0]
        
        # Create output folder for this PDF's tables
        pdf_output_folder = os.path.join(output_base_folder, f"{pdf_name}-Tables")
        os.makedirs(pdf_output_folder, exist_ok=True)

        print(f"\nProcessing PDF: {pdf_file}")
        
        # Open PDF
        pdf_document = fitz.open(pdf_path)
        
        # Dictionary to count tables per page
        table_counter = {}

        # Process each page
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            
            # Convert PDF page to image
            pix = page.get_pixmap(dpi=dpi, colorspace=fitz.csRGB)
            
            # Convert to numpy array for OpenCV
            img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, 3)
            
            # Convert RGB to BGR for OpenCV
            image_cv = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
            
            page_name = f'page_{page_number + 1}'
            
            # Create a temporary file for the image
            temp_image_path = os.path.join(output_base_folder, "temp_page.png")
            cv2.imwrite(temp_image_path, image_cv)
            
            try:
                # Detect tables using Unstructured's partition_image with filename
                elements = partition_image(filename=temp_image_path, infer_table_structure=True, strategy='hi_res')
                
                # Convert elements to JSON structure
                element_dict = [el.to_dict() for el in elements]
                
                # Extract and crop tables
                for item in element_dict:
                    if isinstance(item, dict) and item.get("type") == "Table":
                        try:
                            coordinates = item["metadata"]["coordinates"]["points"]
                            
                            # Count tables per page
                            table_counter[page_name] = table_counter.get(page_name, 0) + 1
                            table_number = table_counter[page_name]
                            
                            # Set output filename for cropped table
                            filename = f"{page_name}_Table_{table_number}.png"
                            
                            # Crop and save the table
                            crop_with_padding(image_cv, coordinates, pdf_output_folder, filename)
                            
                        except KeyError as e:
                            print(f"Missing key {e} in item: {item}")
            
            finally:
                # Clean up temporary file
                if os.path.exists(temp_image_path):
                    os.remove(temp_image_path)
            
            print(f"Processed page {page_number + 1}/{pdf_document.page_count}")
        
        pdf_document.close()
        print(f"Finished processing PDF: {pdf_file}! Tables saved in: {pdf_output_folder}")

    print(f"\nAll PDFs processed successfully! Extracted tables are stored in: {output_base_folder}")

# Example usage:
input_directory = "TEST INPUT PDF"  # Directory containing PDF files
output_directory = "TEST RESULT3/Detected Tables"  # Directory for output

crop_tables_from_pdf(input_directory, output_directory)

Ocr Extraction and Recognition to save as csv - Accepted -- Done

In [ ]:
def ocr_restructure_tables_as_csv(base_input_folder, output_base_folder="TEST RESULT6/Extracted Tables CSV"):
    """
    Extract tables from images in all subfolders and save as CSV files.
    
    Args:
        base_input_folder (str): Path to 'Extracted Tables' folder containing multiple input folders
        output_base_folder (str): Path to save output CSV files
    """
    # Create base output folder if it doesn't exist
    os.makedirs(output_base_folder, exist_ok=True)
    
    ocr = PaddleOCR(lang='en')

    def intersection(box_1, box_2):
        return [box_2[0], box_1[1], box_2[2], box_1[3]]

    def iou(box_1, box_2):
        x_1 = max(box_1[0], box_2[0])
        y_1 = max(box_1[1], box_2[1])
        x_2 = min(box_1[2], box_2[2])
        y_2 = min(box_1[3], box_2[3])

        inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
        if inter == 0:
            return 0

        box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
        box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

        return inter / float(box_1_area + box_2_area - inter)

    # Get all subfolders in the base input folder
    input_folders = [os.path.join(base_input_folder, d) for d in os.listdir(base_input_folder) 
                    if os.path.isdir(os.path.join(base_input_folder, d))]
    
    print(f"Found {len(input_folders)} folders to process")

    for input_folder in input_folders:
        folder_name = os.path.basename(input_folder)
        pdf_name = folder_name.replace('-Tables', '')  # Remove '-Tables' suffix
        
        # Create PDF-specific output folder with -csv suffix
        output_folder = os.path.join(output_base_folder, f"{pdf_name}-csv")
        os.makedirs(output_folder, exist_ok=True)
        
        print(f"\nProcessing folder: {folder_name}")
        
        # Get all image files in current folder
        image_files = [f for f in os.listdir(input_folder) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        print(f"Found {len(image_files)} images")

        for image_file in image_files:
            image_path = os.path.join(input_folder, image_file)
            image_cv = cv2.imread(image_path)

            if image_cv is None:
                print(f"Error loading image: {image_path}")
                continue

            print(f"Processing: {image_file}")
            image_height, image_width = image_cv.shape[:2]
            output = ocr.ocr(image_path)[0]

            if not output:
                print(f"No OCR output for image: {image_file}")
                continue

            # Extract bounding boxes and text
            boxes = [line[0] for line in output]
            texts = [line[1][0] for line in output]
            probabilities = [line[1][1] for line in output]

            # Generate horizontal and vertical boxes
            horiz_boxes = []
            vert_boxes = []

            for box in boxes:
                x_h, x_v = 0, int(box[0][0])
                y_h, y_v = int(box[0][1]), 0
                width_h, width_v = image_width, int(box[2][0] - box[0][0])
                height_h, height_v = int(box[2][1] - box[0][1]), image_height

                horiz_boxes.append([x_h, y_h, x_h + width_h, y_h + height_h])
                vert_boxes.append([x_v, y_v, x_v + width_v, y_v + height_v])

            # Apply NMS
            horiz_out = tf.image.non_max_suppression(
                horiz_boxes, probabilities, max_output_size=1000, iou_threshold=0.1
            )
            vert_out = tf.image.non_max_suppression(
                vert_boxes, probabilities, max_output_size=1000, iou_threshold=0.1
            )

            horiz_lines = np.sort(np.array(horiz_out))
            vert_lines = np.sort(np.array(vert_out))

            # Create table structure
            out_array = [["" for _ in range(len(vert_lines))] for _ in range(len(horiz_lines))]
            unordered_boxes = [vert_boxes[i][0] for i in vert_lines]
            ordered_boxes = np.argsort(unordered_boxes)

            # Fill table with text
            for i in range(len(horiz_lines)):
                for j in range(len(vert_lines)):
                    resultant = intersection(
                        horiz_boxes[horiz_lines[i]], 
                        vert_boxes[vert_lines[ordered_boxes[j]]]
                    )

                    for b in range(len(boxes)):
                        the_box = [boxes[b][0][0], boxes[b][0][1], boxes[b][2][0], boxes[b][2][1]]
                        if iou(resultant, the_box) > 0.1:
                            out_array[i][j] = texts[b]

            out_array = np.array(out_array)

            # Save as CSV
            csv_filename = f"{os.path.splitext(image_file)[0]}.csv"
            csv_path = os.path.join(output_folder, csv_filename)
            pd.DataFrame(out_array).to_csv(csv_path, index=False, header=False)
            print(f"Saved CSV: {csv_filename} in {output_folder}")

    print(f"\nProcessing complete! CSV files saved in: {output_base_folder}")

# Example usage:
base_input_folder = "TEST RESULT4/Extracted Tables"  # Path to folder containing multiple table folders
ocr_restructure_tables_as_csv(base_input_folder)

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('PDF1 Tables/budget_speech-Pages-Cropped/page_40_Table_1.csv')
df1

In [ ]:
df2 = pd.read_csv('PDF1 Tables/budget_speech-Pages-Cropped\page_31_Table_1.csv')
df2

Crop and page text - working nice

In [ ]:
def extract_tables_and_text_from_pdf(input_dir, output_base_folder="Extracted Tables", text_output_folder="Extracted Text", dpi=300, top_left_padding=5, bottom_right_padding=7):
    """
    Extract tables and text directly from PDF files without storing intermediate page images.
    """
    os.makedirs(output_base_folder, exist_ok=True)
    os.makedirs(text_output_folder, exist_ok=True)
    
    # Get all PDF files from input directory
    pdf_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.pdf')]

    # Function to crop an image with padding and save it
    def crop_with_padding(image, coordinates, output_folder, filename):
        """Crop the specified coordinates from the image, apply padding, and save it."""
        x_min = int(min(pt[0] for pt in coordinates))
        y_min = int(min(pt[1] for pt in coordinates))
        x_max = int(max(pt[0] for pt in coordinates))
        y_max = int(max(pt[1] for pt in coordinates))

        # Apply top-left and bottom-right padding
        x_min_padded = max(0, x_min - top_left_padding)
        y_min_padded = max(0, y_min - top_left_padding)
        x_max_padded = min(image.shape[1], x_max + bottom_right_padding * 2)
        y_max_padded = min(image.shape[0], y_max + bottom_right_padding)

        # Crop the image
        cropped_image = image[y_min_padded:y_max_padded, x_min_padded:x_max_padded]

        # Save cropped image
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, cropped_image)
        print(f"Cropped table saved to: {output_path}")

    # Process each PDF file
    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_dir, pdf_file)
        pdf_name = os.path.splitext(pdf_file)[0]
        
        # Create output folder for this PDF's tables and text
        pdf_output_folder = os.path.join(output_base_folder, f"{pdf_name}-Tables")
        pdf_text_output_folder = os.path.join(text_output_folder, f"{pdf_name}-Texts")
        os.makedirs(pdf_output_folder, exist_ok=True)
        os.makedirs(pdf_text_output_folder, exist_ok=True)

        print(f"\nProcessing PDF: {pdf_file}")
        
        # Open PDF
        pdf_document = fitz.open(pdf_path)
        
        # Dictionary to count tables per page
        table_counter = {}

        # Process each page
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            
            # Convert PDF page to image
            pix = page.get_pixmap(dpi=dpi, colorspace=fitz.csRGB)
            
            # Convert to numpy array for OpenCV
            img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, 3)
            
            # Convert RGB to BGR for OpenCV
            image_cv = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
            
            page_name = f'page_{page_number + 1}'
            
            # Create a temporary file for the image
            temp_image_path = os.path.join(output_base_folder, "temp_page.png")
            cv2.imwrite(temp_image_path, image_cv)
            
            try:
                # Detect tables using Unstructured's partition_image with filename
                elements = partition_image(filename=temp_image_path, infer_table_structure=True, strategy='hi_res')
                
                # Convert elements to JSON structure
                element_dict = [el.to_dict() for el in elements]
                
                # Extract and crop tables or extract text
                for item in element_dict:
                    if isinstance(item, dict):
                        if item.get("type") == "Table":
                            try:
                                coordinates = item["metadata"]["coordinates"]["points"]
                                
                                # Count tables per page
                                table_counter[page_name] = table_counter.get(page_name, 0) + 1
                                table_number = table_counter[page_name]
                                
                                # Set output filename for cropped table
                                filename = f"{page_name}_Table_{table_number}.png"
                                
                                # Crop and save the table
                                crop_with_padding(image_cv, coordinates, pdf_output_folder, filename)
                                
                            except KeyError as e:
                                print(f"Missing key {e} in item: {item}")
                        else:
                            # Extract text if type is not "Table"
                            text_content = item.get("text", "")
                            if text_content:
                                text_filename = f"{page_name}_text.txt"
                                text_output_path = os.path.join(pdf_text_output_folder, text_filename)
                                with open(text_output_path, "a", encoding="utf-8") as text_file:
                                    text_file.write(text_content + "\n")
                                print(f"Text extracted and saved to: {text_output_path}")
            
            finally:
                # Clean up temporary file
                if os.path.exists(temp_image_path):
                    os.remove(temp_image_path)
            
            print(f"Processed page {page_number + 1}/{pdf_document.page_count}")
        
        pdf_document.close()
        print(f"Finished processing PDF: {pdf_file}! Tables saved in: {pdf_output_folder}, Text saved in: {pdf_text_output_folder}")

    print(f"\nAll PDFs processed successfully! Extracted tables are stored in: {output_base_folder}, Extracted text is stored in: {text_output_folder}")

# Example usage:
input_directory = "TEST INPUT PDF"  # Directory containing PDF files
output_directory = "TEST RESULT10/Extracted Tables"  # Directory for output tables
text_output_directory = "TEST RESULT10/Extracted Text"  # Directory for output text

extract_tables_and_text_from_pdf(input_directory, output_directory, text_output_directory)

Combined Process 1.0 

In [ ]:
def process_pdf_documents(input_dir, output_base_dir="TEST RESULT4", dpi=300):
    """
    Process PDFs to extract both text and tables in a single pass.
    
    Args:
        input_dir (str): Directory containing PDF files
        output_base_dir (str): Base directory for outputs
        dpi (int): DPI for PDF to image conversion
    """
    # Initialize output directories
    text_output_folder = os.path.join(output_base_dir, "Extracted Text")
    tables_output_folder = os.path.join(output_base_dir, "Extracted Tables")
    tables_csv_folder = os.path.join(output_base_dir, "Extracted Tables CSV")
    
    for folder in [text_output_folder, tables_output_folder, tables_csv_folder]:
        os.makedirs(folder, exist_ok=True)

    def process_element(element, page_name, pdf_name, image_cv):
        """Process individual elements (text or table) from the page"""
        if element.get("type") == "Table":
            # Handle table element
            try:
                coordinates = element["metadata"]["coordinates"]["points"]
                table_folder = os.path.join(tables_output_folder, f"{pdf_name}-Tables")
                os.makedirs(table_folder, exist_ok=True)
                
                # Crop and save table image
                x_min = int(min(pt[0] for pt in coordinates))
                y_min = int(min(pt[1] for pt in coordinates))
                x_max = int(max(pt[0] for pt in coordinates))
                y_max = int(max(pt[1] for pt in coordinates))
                
                # Add padding
                x_min = max(0, x_min - 5)
                y_min = max(0, y_min - 5)
                x_max = min(image_cv.shape[1], x_max + 14)
                y_max = min(image_cv.shape[0], y_max + 7)
                
                cropped_table = image_cv[y_min:y_max, x_min:x_max]
                table_filename = f"{page_name}_Table_{len(os.listdir(table_folder)) + 1}.png"
                cv2.imwrite(os.path.join(table_folder, table_filename), cropped_table)
                
                # Process table with OCR
                ocr_result = ocr.ocr(os.path.join(table_folder, table_filename))[0]
                if ocr_result:
                    process_table_ocr(ocr_result, table_filename, tables_csv_folder)
                
            except (KeyError, IndexError) as e:
                print(f"Error processing table: {e}")
                
        elif element.get("type") != "Table":
            # Handle text element
            text_content = element.get("text", "")
            if text_content:
                text_folder = os.path.join(text_output_folder, f"{pdf_name}-Texts")
                os.makedirs(text_folder, exist_ok=True)
                text_filename = f"{page_name}_text.txt"
                text_path = os.path.join(text_folder, text_filename)
                
                with open(text_path, "a", encoding="utf-8") as text_file:
                    text_file.write(text_content + "\n")

    def process_table_ocr(ocr_result, table_filename, output_folder):
        """Process OCR results and save as CSV"""
        boxes = [line[0] for line in ocr_result]
        texts = [line[1][0] for line in ocr_result]
        probabilities = [line[1][1] for line in ocr_result]
        
        # Generate horizontal and vertical boxes
        image_height, image_width = 1000, 1000  # Default size
        horiz_boxes = []
        vert_boxes = []
        
        for box in boxes:
            x_h, x_v = 0, int(box[0][0])
            y_h, y_v = int(box[0][1]), 0
            width_h, width_v = image_width, int(box[2][0] - box[0][0])
            height_h, height_v = int(box[2][1] - box[0][1]), image_height
            
            horiz_boxes.append([x_h, y_h, x_h + width_h, y_h + height_h])
            vert_boxes.append([x_v, y_v, x_v + width_v, y_v + height_v])
        
        # Apply NMS and create table structure
        horiz_out = tf.image.non_max_suppression(
            horiz_boxes, probabilities, max_output_size=1000, iou_threshold=0.1
        )
        vert_out = tf.image.non_max_suppression(
            vert_boxes, probabilities, max_output_size=1000, iou_threshold=0.1
        )
        
        horiz_lines = np.sort(np.array(horiz_out))
        vert_lines = np.sort(np.array(vert_out))
        
        # Create and fill table structure
        out_array = [["" for _ in range(len(vert_lines))] for _ in range(len(horiz_lines))]
        unordered_boxes = [vert_boxes[i][0] for i in vert_lines]
        ordered_boxes = np.argsort(unordered_boxes)
        
        for i in range(len(horiz_lines)):
            for j in range(len(vert_lines)):
                resultant = [
                    vert_boxes[vert_lines[ordered_boxes[j]]][0],
                    horiz_boxes[horiz_lines[i]][1],
                    vert_boxes[vert_lines[ordered_boxes[j]]][2],
                    horiz_boxes[horiz_lines[i]][3]
                ]
                
                for b in range(len(boxes)):
                    the_box = [boxes[b][0][0], boxes[b][0][1], boxes[b][2][0], boxes[b][2][1]]
                    if calculate_iou(resultant, the_box) > 0.1:
                        out_array[i][j] = texts[b]
        
        # Save as CSV
        csv_filename = f"{os.path.splitext(table_filename)[0]}.csv"
        csv_path = os.path.join(output_folder, csv_filename)
        pd.DataFrame(out_array).to_csv(csv_path, index=False, header=False)

    def calculate_iou(box_1, box_2):
        """Calculate Intersection over Union"""
        x_1 = max(box_1[0], box_2[0])
        y_1 = max(box_1[1], box_2[1])
        x_2 = min(box_1[2], box_2[2])
        y_2 = min(box_1[3], box_2[3])
        
        inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
        if inter == 0:
            return 0
            
        box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
        box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))
        
        return inter / float(box_1_area + box_2_area - inter)

    # Initialize OCR
    ocr = PaddleOCR(lang='en')
    
    # Process each PDF
    pdf_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.pdf')]
    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_dir, pdf_file)
        pdf_name = os.path.splitext(pdf_file)[0]
        print(f"\nProcessing PDF: {pdf_file}")
        
        pdf_document = fitz.open(pdf_path)
        
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            page_name = f'page_{page_number + 1}'
            
            # Convert PDF page to image
            pix = page.get_pixmap(dpi=dpi, colorspace=fitz.csRGB)
            img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, 3)
            image_cv = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
            
            # Create temporary image file
            temp_image_path = os.path.join(output_base_dir, "temp_page.png")
            cv2.imwrite(temp_image_path, image_cv)
            
            try:
                # Extract elements using Unstructured
                elements = partition_image(filename=temp_image_path, 
                                        infer_table_structure=True, 
                                        strategy='hi_res')
                element_dict = [el.to_dict() for el in elements]
                
                # Process each element
                for element in element_dict:
                    process_element(element, page_name, pdf_name, image_cv)
                    
            finally:
                if os.path.exists(temp_image_path):
                    os.remove(temp_image_path)
            
            print(f"Processed page {page_number + 1}/{pdf_document.page_count}")
        
        pdf_document.close()
        print(f"Finished processing PDF: {pdf_file}")

    print("\nProcessing complete!")
    print(f"Text extracted to: {text_output_folder}")
    print(f"Tables extracted to: {tables_output_folder}")
    print(f"Table CSVs saved to: {tables_csv_folder}")

# Example usage:
input_directory = "TEST INPUT PDF"
process_pdf_documents(input_directory)

Combined Process 1.1 - csv path correction

In [ ]:
def process_pdf_documents_update(input_dir, output_base_dir="TEST RESULT17", dpi=300):
    """
    Process PDFs to extract both text and tables in a single pass.
    
    Args:
        input_dir (str): Directory containing PDF files
        output_base_dir (str): Base directory for outputs
        dpi (int): DPI for PDF to image conversion
    """
    # Initialize output directories
    text_output_folder = os.path.join(output_base_dir, "Extracted Text")
    tables_output_folder = os.path.join(output_base_dir, "Extracted Tables")
    tables_csv_folder = os.path.join(output_base_dir, "Extracted Tables CSV")
    
    for folder in [text_output_folder, tables_output_folder, tables_csv_folder]:
        os.makedirs(folder, exist_ok=True)

    def intersection(box_1, box_2):
        return [box_2[0], box_1[1], box_2[2], box_1[3]]

    def iou(box_1, box_2):
        x_1 = max(box_1[0], box_2[0])
        y_1 = max(box_1[1], box_2[1])
        x_2 = min(box_1[2], box_2[2])
        y_2 = min(box_1[3], box_2[3])

        inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
        if inter == 0:
            return 0

        box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
        box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

        return inter / float(box_1_area + box_2_area - inter)

    def process_element(element, page_name, pdf_name, image_cv):
        """Process individual elements (text or table) from the page"""
        if element.get("type") == "Table":
            try:
                coordinates = element["metadata"]["coordinates"]["points"]
                table_folder = os.path.join(tables_output_folder, f"{pdf_name}-Tables")
                csv_folder = os.path.join(tables_csv_folder, f"{pdf_name}-csv")
                
                # Create necessary folders
                os.makedirs(table_folder, exist_ok=True)
                os.makedirs(csv_folder, exist_ok=True)
                
                # Crop and save table image
                x_min = int(min(pt[0] for pt in coordinates))
                y_min = int(min(pt[1] for pt in coordinates))
                x_max = int(max(pt[0] for pt in coordinates))
                y_max = int(max(pt[1] for pt in coordinates))
                
                # Add padding
                x_min = max(0, x_min - 5)
                y_min = max(0, y_min - 5)
                x_max = min(image_cv.shape[1], x_max + 14)
                y_max = min(image_cv.shape[0], y_max + 7)
                
                cropped_table = image_cv[y_min:y_max, x_min:x_max]
                table_filename = f"{page_name}_Table_{len(os.listdir(table_folder)) + 1}.png"
                table_path = os.path.join(table_folder, table_filename)
                cv2.imwrite(table_path, cropped_table)
                print(f"Cropped table saved to: {table_path}")
                
                # Process table with OCR and restructure
                output = ocr.ocr(table_path)[0]
                if not output:
                    print(f"No OCR output for table: {table_filename}")
                    return

                # Extract bounding boxes and text
                boxes = [line[0] for line in output]
                texts = [line[1][0] for line in output]
                probabilities = [line[1][1] for line in output]

                # Generate horizontal and vertical boxes
                image_height, image_width = cropped_table.shape[:2]
                horiz_boxes = []
                vert_boxes = []

                for box in boxes:
                    x_h, x_v = 0, int(box[0][0])
                    y_h, y_v = int(box[0][1]), 0
                    width_h, width_v = image_width, int(box[2][0] - box[0][0])
                    height_h, height_v = int(box[2][1] - box[0][1]), image_height

                    horiz_boxes.append([x_h, y_h, x_h + width_h, y_h + height_h])
                    vert_boxes.append([x_v, y_v, x_v + width_v, y_v + height_v])

                # Apply NMS
                horiz_out = tf.image.non_max_suppression(
                    horiz_boxes, probabilities, max_output_size=1000, iou_threshold=0.1
                )
                vert_out = tf.image.non_max_suppression(
                    vert_boxes, probabilities, max_output_size=1000, iou_threshold=0.1
                )

                horiz_lines = np.sort(np.array(horiz_out))
                vert_lines = np.sort(np.array(vert_out))

                # Create table structure
                out_array = [["" for _ in range(len(vert_lines))] for _ in range(len(horiz_lines))]
                unordered_boxes = [vert_boxes[i][0] for i in vert_lines]
                ordered_boxes = np.argsort(unordered_boxes)

                # Fill table with text
                for i in range(len(horiz_lines)):
                    for j in range(len(vert_lines)):
                        resultant = intersection(
                            horiz_boxes[horiz_lines[i]], 
                            vert_boxes[vert_lines[ordered_boxes[j]]]
                        )

                        for b in range(len(boxes)):
                            the_box = [boxes[b][0][0], boxes[b][0][1], boxes[b][2][0], boxes[b][2][1]]
                            if iou(resultant, the_box) > 0.1:
                                out_array[i][j] = texts[b]

                # Save as CSV
                csv_filename = f"{os.path.splitext(table_filename)[0]}.csv"
                csv_path = os.path.join(csv_folder, csv_filename)
                pd.DataFrame(out_array).to_csv(csv_path, index=False, header=False)
                print(f"Saved CSV: {csv_filename} in {csv_folder}")
                
            except (KeyError, IndexError) as e:
                print(f"Error processing table: {e}")
                
        elif element.get("type") != "Table":
            # Handle text element
            text_content = element.get("text", "")
            if text_content:
                text_folder = os.path.join(text_output_folder, f"{pdf_name}-Texts")
                os.makedirs(text_folder, exist_ok=True)
                text_filename = f"{page_name}_text.txt"
                text_path = os.path.join(text_folder, text_filename)
                
                with open(text_path, "a", encoding="utf-8") as text_file:
                    text_file.write(text_content + "\n")

    # Initialize OCR
    ocr = PaddleOCR(lang='en')
    
    # Process each PDF
    pdf_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.pdf')]
    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_dir, pdf_file)
        pdf_name = os.path.splitext(pdf_file)[0]
        print(f"\nProcessing PDF: {pdf_file}")
        
        pdf_document = fitz.open(pdf_path)
        
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            page_name = f'page_{page_number + 1}'
            
            # Convert PDF page to image
            pix = page.get_pixmap(dpi=dpi, colorspace=fitz.csRGB)
            img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, 3)
            image_cv = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
            
            # Create temporary image file
            temp_image_path = os.path.join(output_base_dir, "temp_page.png")
            cv2.imwrite(temp_image_path, image_cv)
            
            try:
                # Extract elements using Unstructured
                elements = partition_image(filename=temp_image_path, 
                                        infer_table_structure=True, 
                                        strategy='hi_res')
                element_dict = [el.to_dict() for el in elements]
                
                # Process each element
                for element in element_dict:
                    process_element(element, page_name, pdf_name, image_cv)
                    
            finally:
                if os.path.exists(temp_image_path):
                    os.remove(temp_image_path)
            
            print(f"Processed page {page_number + 1}/{pdf_document.page_count}")
        
        pdf_document.close()
        print(f"Finished processing PDF: {pdf_file}")

    print("\nProcessing complete!")
    print(f"Text extracted to: {text_output_folder}")
    print(f"Tables extracted to: {tables_output_folder}")
    print(f"Table CSVs saved to: {tables_csv_folder}")

# Example usage:
input_directory = "inputs1"
process_pdf_documents_update(input_directory)